## Etherscan Notebook 

Looking at how to derive price for tokens and gas fees by pool.  Pool example can be found [here](https://app.uniswap.org/explore/pools/ethereum/0x88e6A0c2dDD26FEEb64F039a2c41296FcB3f5640)

In [2]:
import requests
import time 
import os
import pandas as pd
import numpy as np
from web3 import Web3

In [3]:
# change the active path to the parent directory 
if True: 
    print("Moving active path to parent directory")
    os.chdir('..')
    print(os.getcwd())

Moving active path to parent directory
/Users/das/DATASCI210/arbitrage_playground


In [4]:
import src.arbutils as arbutils

In [5]:
API_KEY =  os.getenv('GRAPH_API_KEY')
API_KEY2 =  os.getenv('ETHERSCAN_API_KEY')

if not API_KEY:
    print("No GRAPH_API_KEY found")
else:
    print("Found GRAPH API Key!")


if not API_KEY2:
    print("No ETHERSCAN_API_KEY found")
else:
    print("Found ETHERSCAN API Key!")

Found GRAPH API Key!
Found ETHERSCAN API Key!


In [6]:
POOL0_ADDRESS="0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640" # USDC / WETH (0.05%) 
POOL0_TXN_FEE = 0.0005
POOL1_ADDRESS="0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8" # USDC / WETH (0.3%)
POOl1_TXN_FEE = 0.003

GWEI_SCALER = 1e9

In [7]:
p0 = arbutils.thegraph_request(API_KEY, 
                            POOL0_ADDRESS,
                            new_date=None, 
                            old_date=None, 
                            data_path=None, 
                            checkpoint_file=None)

p1 = arbutils.thegraph_request(API_KEY, 
                            POOL1_ADDRESS,
                            new_date=None, 
                            old_date=None, 
                            data_path=None, 
                            checkpoint_file=None)

merged_pools = arbutils.merge_pool_data_v2(p0, 
                                           POOL0_TXN_FEE, 
                                           p1, 
                                           POOl1_TXN_FEE)


Starting from timestamp: 1736898871
query number 0 1736898871 1736902471
Batch 1, Total swaps: 264, Avg Fetch Time: 0.5s
query number 1 1736902463 1736902471
Starting from timestamp: 1736898871
query number 0 1736898871 1736902471
Batch 1, Total swaps: 9, Avg Fetch Time: 0.2s
query number 1 1736901407 1736902471
Are there any NaNs in the DataFrame? False


/Users/das/DATASCI210/arbitrage_playground/src/arbutils.py:540: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  both_pools = both_pools.ffill().reset_index(drop=True)


In [8]:
p0.columns

Index(['transactionHash', 'datetime', 'timeStamp', 'sqrtPriceX96',
       'blockNumber', 'gasPrice', 'gasUsed', 'tick', 'amount0', 'amount1',
       'liquidity'],
      dtype='object')

In [9]:
p0.head()

,transactionHash,datetime,timeStamp,sqrtPriceX96,blockNumber,gasPrice,gasUsed,tick,amount0,amount1,liquidity
0,0xdfe8deaecde582e9da2e906d1eef85c6d388764d422d...,2025-01-14 23:54:59+00:00,1736898899,1.395327e+33,21626136,3.780745e+09,0.0,195535.0,-9.990000,0.003100,-1.0
1,0x464b2f8c761ef279c897ecba0a261d4f5285791573f8...,2025-01-14 23:55:47+00:00,1736898947,1.395233e+33,21626140,5.601803e+09,0.0,195534.0,3000.000000,-0.929916,-1.0
2,0xc00b302ee9816178fa9150feecd8681e94095ad15771...,2025-01-14 23:55:47+00:00,1736898947,1.395251e+33,21626140,5.601803e+09,0.0,195534.0,12636.350439,-3.917180,-1.0
3,0x95e68d5b2272b5cc793ee5cb37e506fc70342e2695af...,2025-01-14 23:56:23+00:00,1736898983,1.395158e+33,21626143,5.859279e+09,0.0,195533.0,12381.371675,-3.837624,-1.0
4,0xe45a872f586eac91ee9af049ea64d13826a85fd3ce78...,2025-01-14 23:56:47+00:00,1736899007,1.395158e+33,21626145,5.545332e+09,0.0,195533.0,-15.975276,0.004956,-1.0


Now that I have information at the transaction level.  I would like to parse specific transactions to extract the gasUsed value which is the only value missing for our current model.   Below we will fetch token transfers based on the blocks that we have in the data frame above.

In [13]:
def fetch_tokentx_data(etherscan_api_key, pool_address, start_block=0, end_block=99999999):

    base_url = 'https://api.etherscan.io/api'

    params = {
        'module': 'account',
        'action': 'tokentx',
        'address': pool_address,
        'startblock': start_block,
        'endblock': end_block,
        'sort': 'desc',
        'apikey': etherscan_api_key
    }
    
    
    response = requests.get(base_url, params=params)
    if response.status_code != 200:
        #st.error(f"API request failed with status code {response.status_code}")
        raise Exception(f"API request failed with status code {response.status_code}")
    
    data = response.json()
    if data['status'] != '1':
        #st.error(f"API returned an error: {data['result']}")
        raise Exception(f"API returned an error: {data['result']}")
    
    df = pd.DataFrame(data['result'])
    
    expected_columns = ['hash', 'blockNumber', 'timeStamp', 'from', 'to', 'gas', 'gasPrice', 'gasUsed', 'cumulativeGasUsed', 'confirmations', 'tokenSymbol', 'value', 'tokenName']
    
    for col in expected_columns:
        if col not in df.columns:
            raise Exception(f"Expected column '{col}' is missing from the response")
    
    df.sort_values(by='timeStamp')
    
    consolidated_data = {}

    for index, row in df.iterrows():
        tx_hash = row['hash']
        
        if tx_hash not in consolidated_data:
            consolidated_data[tx_hash] = {
                'blockNumber': np.int32(row['blockNumber']),
                'timeStamp': int(row['timeStamp']),
                'transactionHash': tx_hash,
                'from': row['from'],
                'to': row['to'],
                'WETH_value': 0,
                'USDC_value': 0,
                'tokenName_WETH': '',
                'tokenName_USDC': '',
                'gas': float(row['gas']),
                'gasPrice': float(row['gasPrice']),
                'gasUsed': float(row['gasUsed']),
                'cumulativeGasUsed': float(row['cumulativeGasUsed']),
                'confirmations': row['confirmations']
            }
        
        if row['tokenSymbol'] == 'WETH':
            consolidated_data[tx_hash]['WETH_value'] = float(row['value'])
            consolidated_data[tx_hash]['tokenName_WETH'] = row['tokenName']
        elif row['tokenSymbol'] == 'USDC':
            consolidated_data[tx_hash]['USDC_value'] = float(row['value'])
            consolidated_data[tx_hash]['tokenName_USDC'] = row['tokenName']

    final_df = pd.DataFrame.from_dict(consolidated_data, orient='index').reset_index(drop=True)

    return final_df.sort_values(by='timeStamp')

Here we can access the list of blocks that we need to get the gasUsed values for...

In [65]:
# Example usage
startblock = p0['blockNumber'].iloc[0]
endblock = p0['blockNumber'].iloc[-1]
print(f"Start Block: {startblock}")
print(f"End Block: {endblock}")

Start Block: 21626136
End Block: 21626431


Use the fetch_tokentx_data method to fetch the token transfers in that block range from Etherscan...

In [77]:
p0_tokentx = fetch_tokentx_data(API_KEY2, POOL0_ADDRESS, start_block=startblock, end_block=endblock)
p1_tokentx = fetch_tokentx_data(API_KEY2, POOL1_ADDRESS, start_block=startblock, end_block=endblock)

In [18]:
p0_tokentx.head()

,blockNumber,timeStamp,transactionHash,from,to,WETH_value,USDC_value,tokenName_WETH,tokenName_USDC,gas,gasPrice,gasUsed,cumulativeGasUsed,confirmations
228,21626136,1736898899,0xdfe8deaecde582e9da2e906d1eef85c6d388764d422d...,0x3fc91a3afd70395cd496c647d5a6cc9d4b2b7fad,0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640,3.100098e+15,9.990000e+06,Wrapped Ether,USDC,641134.0,3.780745e+09,229689.0,17995123.0,310
227,21626140,1736898947,0xc00b302ee9816178fa9150feecd8681e94095ad15771...,0x85ccdcf580caf8a3674d94f3d6c63304674a8e09,0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640,3.917180e+18,1.263635e+10,Wrapped Ether,USDC,320140.0,5.601803e+09,250214.0,1910974.0,306
226,21626140,1736898947,0x464b2f8c761ef279c897ecba0a261d4f5285791573f8...,0xb355a5db651ba12f3fcc18529f140a9597616579,0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640,9.299161e+17,3.000000e+09,Wrapped Ether,USDC,340382.0,5.601803e+09,249237.0,2290443.0,306
225,21626142,1736898971,0x4714b333010385e3154bfeb62068031835bc6cd3e5ad...,0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640,0xc36442b4a4522e871399cd717abdd847ab11fe88,1.388647e+12,4.458000e+03,Wrapped Ether,USDC,246194.0,3.804275e+09,194740.0,14411826.0,304
224,21626143,1736898983,0x95e68d5b2272b5cc793ee5cb37e506fc70342e2695af...,0x3fc91a3afd70395cd496c647d5a6cc9d4b2b7fad,0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640,3.837624e+18,1.238137e+10,Wrapped Ether,USDC,405547.0,5.859279e+09,305361.0,10851488.0,303


Below we can see that the token transfer has slightly more blocks available to it. this is because we really only care about swaps in this context and token transfer includes other types of transactions.

In [67]:
print(f"p0 number of unique blocks: {p0.blockNumber.unique().shape[0]}")
print(f"p0_tokentx number of unique blocks: {p0_tokentx.blockNumber.unique().shape[0]}")

p0 number of unique blocks: 157
p0_tokentx number of unique blocks: 161


In [75]:
block_cnt = 0
for block in list(p0['blockNumber'].unique()):
  if block in list(p0_new.blockNumber.unique()):
      block_cnt += 1
print(f"{block_cnt}")

157


In order to merge the data frames we take only what we need from p0_tokentx (i.e. gasUsed). So we drop 'gasUsed' from p0 and we only merge 'gasUsed' from p0_token.  We can see that we end up with the same number of blocks and we can see that the p0_new has now integrated the missing column.

In [78]:
p1_new = pd.merge(p1.drop(labels=['gasUsed'],axis=1), p1_tokentx[['timeStamp','blockNumber','gasUsed']], on=['timeStamp','blockNumber'], how='left')
p0_new = pd.merge(p0.drop(labels=['gasUsed'],axis=1), p0_tokentx[['timeStamp','blockNumber','gasUsed']], on=['timeStamp','blockNumber'], how='left')
print(p0_new.blockNumber.unique().shape)
display(p0_new.columns)
p0_new.head()

(157,)


Index(['transactionHash', 'datetime', 'timeStamp', 'sqrtPriceX96',
       'blockNumber', 'gasPrice', 'tick', 'amount0', 'amount1', 'liquidity',
       'gasUsed'],
      dtype='object')

,transactionHash,datetime,timeStamp,sqrtPriceX96,blockNumber,gasPrice,tick,amount0,amount1,liquidity,gasUsed
0,0xdfe8deaecde582e9da2e906d1eef85c6d388764d422d...,2025-01-14 23:54:59+00:00,1736898899,1.395327e+33,21626136,3.780745e+09,195535.0,-9.990000,0.003100,-1.0,229689.0
1,0x464b2f8c761ef279c897ecba0a261d4f5285791573f8...,2025-01-14 23:55:47+00:00,1736898947,1.395233e+33,21626140,5.601803e+09,195534.0,3000.000000,-0.929916,-1.0,250214.0
2,0x464b2f8c761ef279c897ecba0a261d4f5285791573f8...,2025-01-14 23:55:47+00:00,1736898947,1.395233e+33,21626140,5.601803e+09,195534.0,3000.000000,-0.929916,-1.0,249237.0
3,0xc00b302ee9816178fa9150feecd8681e94095ad15771...,2025-01-14 23:55:47+00:00,1736898947,1.395251e+33,21626140,5.601803e+09,195534.0,12636.350439,-3.917180,-1.0,250214.0
4,0xc00b302ee9816178fa9150feecd8681e94095ad15771...,2025-01-14 23:55:47+00:00,1736898947,1.395251e+33,21626140,5.601803e+09,195534.0,12636.350439,-3.917180,-1.0,249237.0


**Get price within the Pool in ETH/USDC**

To derive the price for the pool in ETH/USDC, you must use the sqrtPriceX96 value, which is the pool price immediately after the transaction takes place (including slippage).  You can see below that there is almost always a descrepency, but its not always enough to over come transaction and gas fees (see below).

In [79]:
# row to pick for the swap...used just for the example.
pool0_price_in_USDC_per_ETH  = ((p0_new["sqrtPriceX96"].iloc[0] / 2**96)**2 / 1e12) **-1
pool1_price_in_USDC_per_ETH  = ((p1_new["sqrtPriceX96"].iloc[0] / 2**96)**2 / 1e12) **-1

print(f"Pool 0 Price in USDC per ETH (at Tx: 0x...{p0_new['transactionHash'].iloc[0][-4:]}): ${pool0_price_in_USDC_per_ETH:.2f}")
print(f"Pool 1 Price in USDC per ETH (at Tx: 0x...{p1_new['transactionHash'].iloc[0][-4:]}): ${pool1_price_in_USDC_per_ETH:.2f}")
print(f"Difference in price: ${pool1_price_in_USDC_per_ETH-pool0_price_in_USDC_per_ETH:.2f}")

Pool 0 Price in USDC per ETH (at Tx: 0x...bb08): $3224.09
Pool 1 Price in USDC per ETH (at Tx: 0x...7cd5): $3227.00
Difference in price: $2.91


**Get gas fees in ETH**

Gas fees for a transaction include all the 'work' done. There is a rate of fee per unit of work (i.e. gasPrice) and then there is the work done (i.e. gasUsed).  gasPrice and gasUsed is in gwei which is 1e9 of an ETH.  so to convert to eth, each value needs to be converted with the 1e9 scaling.

In [80]:
gas_price_eth_tokens_per_unit = int(p0_new['gasPrice'].iloc[0])/GWEI_SCALER
gas_used_units = int(p0_new['gasUsed'].iloc[0]) / GWEI_SCALER
gas_fees_eth_tokens  = gas_price_eth_tokens_per_unit* gas_used_units
gas_fees_usdc_tokens = pool0_price_in_USDC_per_ETH * gas_fees_eth_tokens 
print(f"Gas Price in ETH per unit: {gas_price_eth_tokens_per_unit}")
print(f"Gas Used in GWEI units for Uniswap Transaction: {gas_used_units}")
print(f"Gas fees for this Transaction in ETH: {gas_fees_eth_tokens:.5f} (ETH)")
print(f"Gas fees for this Transaction in USDC: ${gas_fees_usdc_tokens:.2f} (USDC)")

Gas Price in ETH per unit: 3.780744797
Gas Used in GWEI units for Uniswap Transaction: 0.000229689
Gas fees for this Transaction in ETH: 0.00087 (ETH)
Gas fees for this Transaction in USDC: $2.80 (USDC)
